In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import scipy
from scipy.linalg import hadamard
import scipy.signal
import pylab
from scipy.misc import derivative

# Set up helper functions

In [ ]:
def set_walsh_sim(N, n, stepperiod, verify=False):
        """
        N: order of walsh matrix
        n: walsh index to give this stream
        stepperiod: period (2^?), in multiples of self.periodbase FPGA clocks,
        of shortest walsh step. I.e., 2**13 * 2**self.baseperiod * N
        = period of complete cycle in FPGA clocks.
        """
        depth = 2**12
        N_round = int(2**(np.ceil(np.log2(N))))
        walsh_matrix = hadamard(N_round)
        # reformat so 1 means multiply by -1, and 0 means multiply by 1
        walsh_matrix[walsh_matrix == 1]  = 1
        walsh_matrix[walsh_matrix == -1] = -1
        walsh_func = walsh_matrix[n] # a vector of length N_round
        walsh_func_stretch = walsh_func.repeat(2**stepperiod) # a vector of length N_round * 2*step_period
        return walsh_func_stretch

def spectrometer(timestream, sampling_rate, fft_length, num_spectra_integration):
    """
    Create a spectrometer for a given timestream.

    Parameters:
    - timestream: The input time-domain signal.
    - sampling_rate: The sampling rate of the timestream.
    - fft_length: The length of the FFT.
    - num_spectra_integration: The number of spectra to integrate for each result.

    Returns:
    - frequencies: Array of frequencies.
    - integrated_spectra: List of integrated magnitude spectra.
    """
    num_samples = len(timestream)
    num_segments = int(num_samples / (sampling_rate * num_spectra_integration))
    
    frequencies = np.fft.fftfreq(fft_length, d=1/sampling_rate)[:fft_length//2]
    spectra = []
    integrated_spectra = []

    for i in range(num_segments):
        start_index = i * fft_length
        end_index = start_index + fft_length
        segment = timestream[start_index:end_index]
        
        # Perform FFT
        spectrum = np.fft.fft(segment)[:fft_length//2]
        magnitude_spectrum = np.abs(spectrum)
        spectra.append(magnitude_spectrum)
        
    spectra = np.reshape(spectra,(num_segments,-1))
    
    print(spectra.shape)
    # Integrate the spectra
    for i in range(num_segments//num_spectra_integration):
        start_index = (i)*num_spectra_integration 
        end_index = (i+1)*num_spectra_integration 
        integrated_spectra.append(np.sum(spectra[start_index:end_index],axis=1))

    return frequencies, spectra

def correlator(timestream0, timestream1, sampling_rate, fft_length, num_spectra_integration):
    """
    Create a correlator for a given timestream.

    Parameters:
    - timestream0: The first input time-domain signal.
    - timestream1: The second input time-domain signal.   
    - sampling_rate: The sampling rate of the timestream.
    - fft_length: The length of the FFT.
    - num_spectra_integration: The number of spectra to integrate for each result.

    Returns:
    - frequencies: Array of frequencies.
    - integrated_spectra: List of integrated correlation
    """
    num_samples = len(timestream0)
    num_segments = int(num_samples / (sampling_rate * num_spectra_integration))
    
    frequencies = np.fft.fftfreq(fft_length, d=1/sampling_rate)[:fft_length//2]
    integrated_correlations = []

    for i in range(num_segments):
        start_index = i * int(sampling_rate * num_spectra_integration)
        end_index = start_index + fft_length * num_spectra_integration
        segment0 = timestream0[start_index:end_index]
        segment1 = timestream1[start_index:end_index]
        
        # Perform FFT
        spectrum0 = np.fft.fft(segment0)[:fft_length//2]
        spectrum1 = np.fft.fft(segment1)[:fft_length//2]
        correlation = abs(spectrum0*spectrum1)
        
        # Integrate the spectra
        integrated_correlation = np.sum(np.reshape(correlation, (num_spectra_integration, -1)), axis=0)
        integrated_correlations.append(integrated_correlation)

    return frequencies, integrated_correlations

# Code up Equation from Thompson

Let U(t) be an unwanted system response, and U1(t) be the mean residual spurious voltage

In [ ]:
def mrsv(T,M, u_m):
   return(T**m/(2**(math.factorial(M)))*derivative(u_m))

# Try to replicate Emerson ALMA Memo 537 

In [ ]:
walsh_matrix = hadamard(32)
# reformat so 1 means multiply by -1, and 0 means multiply by 1
walsh_matrix[walsh_matrix == 1]  = 1
walsh_matrix[walsh_matrix == -1] = -1

Generate the first 32 walsh functions

In [ ]:
#Generate walsh patterns of a length 8, which could be used for 8 seperate time streams
fig,ax = plt.subplots(32,1)
fig.set_size_inches(18.5, 38.5)
for i in range(32):
    ax[i].plot(walsh_matrix[i], label=i)
    ax[i].set_title('wal('+str(i)+')')
plt.legend()
plt.tight_layout()
plt.show()

oversample the functions by 100 

In [ ]:
#extend the number of samples for the walsh function to match the signal length 
print(walsh_matrix.shape)
period_base = 100
walshextended = np.zeros((32,3200))
print(walshextended.shape)
for i in range(32):
    n= 0
    n2 = period_base
    for j in range(32):
        extended = np.array(walsh_matrix[i][j].repeat(period_base))
        walshextended[i][n:n2] = extended
        n += period_base
        n2 += period_base

In [ ]:
fig,ax = plt.subplots(32,1)
fig.set_size_inches(18.5, 28.5)
for i in range(32):
    ax[i].plot(walshextended[i])
    #ax[i].set_title('wal('+str(i)+')')
    ax[i].set_ylabel('wal('+str(i)+')', rotation=0, labelpad=30, fontsize=15)
plt.tight_layout()
plt.savefig('walsh_first_32.pdf',format='pdf')
plt.show()

reorder by number of zero crossings

In [ ]:
sequency = {}
for i in range(32):
    wal_ind = ((walshextended[i][:-1] * walshextended[i][1:]) < 0).sum()
    sequency[wal_ind] = walshextended[i]
    
    
sorted_walsh = [sequency[key] for key in sorted(sequency)]

sorted_inds = []
for i in range(32):
    wal_ind = ((sorted_walsh[i][:-1] * sorted_walsh[i][1:]) < 0).sum()
    sorted_inds.append(wal_ind)

In [ ]:
fig,ax = plt.subplots(32,1)
fig.set_size_inches(18.5, 28.5)
for i in range(32):
    ax[i].plot(sorted_walsh[i])
    ax[i].set_ylabel('wal('+str(i)+')', rotation=0, labelpad=30, fontsize=15)

plt.tight_layout()
plt.savefig('walsh_first_32_sequency.pdf',format='pdf')

copy the signal so we can time shift 

In [ ]:
copy = sorted_walsh 
doubled = np.concatenate((sorted_walsh, copy), axis=1)
print(doubled.shape)

In [ ]:
fig,ax = plt.subplots(32,1)
fig.set_size_inches(18.5, 38.5)
for i in range(32):
    ax[i].plot(doubled[i])
    ax[i].set_title('wal('+str(i)+')')
plt.tight_layout()
plt.show()

reorder by number of zero crossings

shift by 1% of the shortest element (1), cross correlate, integrate over T

In [ ]:
np.set_printoptions(threshold=np.inf)
crosstalk = np.zeros((32,32))
for i in range(32):
    shifted = doubled[i][1:3201]
    for j in range(32):
        stable = doubled[j][0:3200]
        crossprod = shifted*stable 
        resid = np.sum(crossprod)
        crosstalk[i,j] = resid/3200

In [ ]:
plt.figure(figsize=(20,20))
fig.set_size_inches(18.5, 10.5)
plt.imshow(abs(crosstalk),vmax=.01)
plt.title('Crosstalk in 1% timing slip')
plt.colorbar()
fig.set_size_inches(18.5, 18.5)
plt.savefig('crosstalk_matrix', format='pdf')

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(crosstalk,vmin=.98)
plt.title('Signal Loss in 1% timing slip')
fig.set_size_inches(18.5, 18.5)

In [ ]:
plt.figure(figsize=(10,8))
for i in range(32):
    for j in range(32):
        if i==j:
            plt.title('Sequency vs. Sensitivity Loss',fontsize=20)
            plt.scatter(sorted_inds[i],((1-crosstalk[i][j])*100), c='slateblue')
            plt.xlabel('Sequency',fontsize=15)
            plt.ylabel('Sensitivity Loss (%)', fontsize=15)
plt.savefig('Sensitivity_loss.pdf',format='pdf')

In [ ]:
#RSS Crosstalk
rss = []
for i in range(32):
    summed = 0 
    for j in range(32):
        if i==j:
            pass
        else:
            summed += (crosstalk[i][j])**2
    rss.append(np.sqrt(summed)*100)

In [ ]:
plt.figure(figsize=(10,8))
plt.title('Sequency vs RSS Crosstalk', fontsize=20)
plt.xlabel('Sequency', fontsize=15)
plt.ylabel('RSS Crosstalk (%)', fontsize=15)
plt.scatter(sorted_inds,rss)
plt.savefig('rsscrosstalk.pdf',format='pdf')

# What if the phase switch has an amplitude imbalance?

Instead of 1, -1, assume a 1% amplitude loss along the 180 degree path. no time shift. 

In [ ]:
imbalanced_walsh = np.zeros_like(doubled)

for i in range(32):
    for j in range(len(imbalanced_walsh[i])):
        if doubled[i][j] == 1:
            imbalanced_walsh[i][j] = 1
        if doubled[i][j] == -1:
            imbalanced_walsh[i][j] = -.99

In [ ]:
fig,ax = plt.subplots(32,1)
fig.set_size_inches(18.5, 38.5)
for i in range(32):
    ax[i].plot(imbalanced_walsh[i], label=i)
    ax[i].set_title('wal('+str(sorted_inds[i])+')')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
crosstalkimb = np.zeros((32,32))
for i in range(32):
    shifted = imbalanced_walsh[i][0:3200]
    for j in range(32):
        stable = doubled[j][0:3200]
        crossprod = shifted*stable 
        resid = np.sum(crossprod)
        crosstalkimb[i,j] = resid/3200

In [ ]:
plt.figure(figsize=(20,15))
for i in range(32):
    for j in range(32):
        if i==j:
            plt.title('Sequency vs. Sensitivity Loss')
            plt.scatter(sorted_inds[i],((1-crosstalkimb[i][j])*100))
            plt.xlabel('Sequency')
            plt.ylabel('Sensitivity Loss (%)')

In [ ]:
#RSS Crosstalk
rssimb = []
for i in range(32):
    summed = 0 
    for j in range(32):
        if i==j:
            pass
        else:
            summed += (crosstalkimb[i][j])**2
    rssimb.append(np.sqrt(summed)*100)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(crosstalkimb,  vmax=.001)
plt.title('Crosstalk in amplitude imbalance')
fig.set_size_inches(18.5, 18.5)

In [ ]:
plt.figure(figsize=(20,15))
plt.title('Sequency vs RSS Crosstalk')
plt.xlabel('Sequency')
plt.ylabel('RSS Crosstalk (%)')
plt.scatter(sorted_inds,rssimb)

# How about a varying signal?


Try to replicate ALMA memo 565 appendix

In [ ]:
# Set the parameters
frequency = 0.006  # Frequency of the sine wave in Hz
duration = 1  # Duration of the signal in seconds
sampling_rate = 3200  # Sampling rate in Hz

t = np.arange(0, duration, 1/sampling_rate)

# Generate the sine wave
sine_wave = np.sin(2 * np.pi * frequency * t)

plt.figure(figsize=(20,15))
plt.plot(t, sine_wave)
plt.title(f"Sine Wave with {frequency} Hz Frequency")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()

In [ ]:
fig,ax = plt.subplots(32,1)
fig.set_size_inches(18.5, 38.5)

for i in range(32):
    mod = sine_wave*doubled[i][0:3200]
    ax[i].plot(mod)
    ax[i].set_ylabel('wal('+str(i)+')', rotation=0, labelpad=30, fontsize=15)

    
plt.tight_layout 
plt.show()

In [ ]:
crosstalk_single = []
for i in range(32):
    crosstalk = np.sum(sine_wave*doubled[i][0:3200])
    normed = crosstalk/3200
    crosstalk_single.append(normed)

In [ ]:
plt.figure(figsize=(20,15))
plt.title('Sequency vs Residual Signal After Integration')
plt.xlabel('Sequency')
plt.ylabel('Residual (Normed)')
plt.scatter(sorted_inds, crosstalk_single)

In [ ]:
crosstalk = np.zeros((32,32))
for i in range(32):
    for j in range(32):
        mult1 = doubled[i][0:3200]*sine_wave
        mult2 = doubled[j][0:3200]*sine_wave
        crossprod = mult1*mult2
        resid = np.sum(crossprod)
        crosstalk[i,j] = resid/3200

In [ ]:
plt.figure(figsize=(20,20))
fig.set_size_inches(18.5, 10.5)
plt.imshow(crosstalk)
plt.title('Crosstalk in time Varying Signal')
fig.set_size_inches(18.5, 18.5)

In [ ]:
rss = []
for i in range(32):
    summed = 0 
    for j in range(32):
        if i==j:
            pass
        else:
            summed += (crosstalk[i][j])**2
    rss.append(np.sqrt(summed)*100)

In [ ]:
plt.figure(figsize=(20,15))
plt.title('Sequency vs RSS Crosstalk')
plt.xlabel('Sequency')
plt.ylabel('RSS Crosstalk (%)')
plt.scatter(sorted_inds,rss)

Try to loop through frequencies 

In [ ]:
# Set the parameters
nfreqs = 50 
freqs = np.linspace(0,2,nfreqs)
sigs = []
cross_freq = []


for freq in freqs: 
    i = 0 
    
    frequency = freq  # Frequency of the sine wave in Hz
    duration = 1  # Duration of the signal in seconds
    sampling_rate = 3200  # Sampling rate in Hz

    t = np.arange(0, duration, 1/sampling_rate)

    # Generate the sine wave
    sine_wave = np.sin(2 * np.pi * frequency * t)
    sigs.append(sine_wave)
    
    crosstalk = np.zeros((32,32))
    
    for i in range(32):
        for j in range(32):
            mult1 = doubled[i][0:3200]*sine_wave
            mult2 = doubled[j][0:3200]*sine_wave
            crossprod = mult1*mult2
            resid = np.sum(crossprod)
            crosstalk[i,j] = resid/3200
            
            
    rss = []
    
    for i in range(32):
        summed = 0 
        for j in range(32):
            if i==j:
                pass
            else:
                summed += (crosstalk[i][j])**2
        rss.append(np.sqrt(summed)*100)
    cross_freq.append(rss)
    i+=1


In [ ]:
for i in range(len(sigs)):
    plt.plot(sigs[i])
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(cross_freq)
plt.xlabel('Walsh Sequency')
plt.ylabel('Frequency #')

In [ ]:
rssvfreq = [item[0] for item in cross_freq]

In [ ]:
plt.plot(rssvfreq)

Non Sinusoids

In [ ]:
noise = np.random.normal(loc=0.0, scale=1.0, size=3200)
plt.figure()
plt.plot(noise)
print(np.sum(noise))
plt.figure()
n = noise.size
timestep = 1/sampling_rate
freq = np.fft.fftfreq(n, d=timestep)
plt.plot(freq, np.fft.fft(noise))
crosstalk_single = []
for i in range(32):
    crosstalk = np.sum(noise*doubled[i][0:3200])
    normed = crosstalk/3200
    crosstalk_single.append(normed)

In [ ]:
crosstalk = np.zeros((32,32))
for i in range(32):
    for j in range(32):
        mult1 = doubled[i][0:3200]*noise
        mult2 = doubled[j][0:3200]*noise
        crossprod = mult1*mult2
        resid = np.sum(crossprod)
        crosstalk[i,j] = resid/3200

In [ ]:
plt.figure(figsize=(20,20))
fig.set_size_inches(18.5, 10.5)
plt.imshow(crosstalk)
plt.title('Crosstalk in Random Noise')
fig.set_size_inches(18.5, 18.5)

In [ ]:
rss = []
    
for i in range(32):
    summed = 0 
    for j in range(32):
        if i==j:
            pass
        else:
            summed += (crosstalk[i][j])**2
    rss.append(np.sqrt(summed)*100)


In [ ]:
plt.figure(figsize=(20,15))
plt.title('Sequency vs RSS Crosstalk')
plt.xlabel('Sequency')
plt.ylabel('RSS Crosstalk (%)')
plt.scatter(sorted_inds,rss)

# Let's do a more realistic sim, with an FFT and HERA bandwidths

In [ ]:
import numpy as np

def pfb_fir_frontend(x, win_coeffs, M, P):
    W = int(x.shape[0] / M / P)
    x_p = x.reshape((W*M, P)).T
    h_p = win_coeffs.reshape((M, P)).T
    x_summed = np.zeros((P, M * W - M))
    for t in range(0, M*W-M):
        x_weighted = x_p[:, t:t+M] * h_p
        x_summed[:, t] = x_weighted.sum(axis=1)
    return x_summed.T

def generate_win_coeffs(M, P, window_fn="hamming"):
    win_coeffs = scipy.signal.get_window(window_fn, M*P)
    sinc       = scipy.signal.firwin(M * P, cutoff=1.0/P, window="rectangular")
    win_coeffs *= sinc
    return win_coeffs

def fft(x_p, P, axis=1):
    return np.fft.rfft(x_p, P, axis=axis)

def pfb_filterbank(x, win_coeffs, M, P):
    x_fir = pfb_fir_frontend(x, win_coeffs, M, P)
    x_pfb = fft(x_fir, P)
    return x_pfb

def pfb_spectrometer(x, n_taps, n_chan, n_int, window_fn="hamming"):
    M = n_taps
    P = n_chan
    
    # Generate window coefficients
    win_coeffs = generate_win_coeffs(M, P, window_fn)

    # Apply frontend, take FFT, then take power (i.e. square)
    x_fir = pfb_fir_frontend(x, win_coeffs, M, P)
    x_pfb = fft(x_fir, P)
    x_psd = np.abs(x_pfb)**2
    
    # Trim array so we can do time integration
    x_psd = x_psd[:np.round(x_psd.shape[0]//n_int)*n_int]
    
    # Integrate over time, by reshaping and summing over axis (efficient)
    x_psd = x_psd.reshape(x_psd.shape[0]//n_int, n_int, x_psd.shape[1])
    x_psd = x_psd.mean(axis=1)
    
    return x_psd

def db(x):
    """ Convert linear value to dB value """
    return 10*np.log10(x)

# What does white noise look like?

In [ ]:
from scipy.signal import butter, lfilter
from numpy.fft import fft, fftfreq

# Parameters
sample_rate = 2000  # Sampling rate in MHz
duration = 1.0  # Duration in seconds
freq_range = (70, 100)  # Frequency range in MHz

# Generate time vector
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)

# Generate white noise
white_noise = np.random.normal(0, 1, len(t))

# Design a bandpass filter
nyquist = 0.5 * sample_rate
low = freq_range[0] / nyquist
high = freq_range[1] / nyquist
b, a = butter(4, [low, high], btype='band')


band_limited_noise = lfilter(b, a, white_noise)

# Compute FFT for the original white noise
fft_white_noise = fft(white_noise)
freqs_white_noise = fftfreq(len(t), 1 / sample_rate)

# Compute FFT for the band-limited white noise
fft_band_limited_noise = fft(band_limited_noise)
freqs_band_limited_noise = fftfreq(len(t), 1 / sample_rate)

# Plotting
plt.figure(figsize=(14, 10))

plt.subplot(3, 2, 1)
plt.plot(t, white_noise)
plt.title('White Noise Time Series')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.subplot(3, 2, 2)
plt.plot(freqs_white_noise, np.abs(fft_white_noise))
plt.title('Frequency Spectrum (Original White Noise)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Amplitude')
plt.xlim(0, sample_rate / 2)

plt.subplot(3, 2, 3)
plt.plot(t, band_limited_noise)
plt.title('Band-Limited White Noise Time Series')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.subplot(3, 2, 4)
plt.plot(freqs_band_limited_noise, np.abs(fft_band_limited_noise))
plt.title('Frequency Spectrum (Band-Limited White Noise)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Amplitude')
plt.xlim(0, sample_rate / 2)

plt.tight_layout()
plt.show()

In [ ]:
M     = 4          # Number of taps
P     = 1024       # Number of 'branches', also fft length
W     = 100       # Number of windows of length M*P in input time stream
n_int = 1          # Number of time integrations on output data


sample_rate = 500  # Sampling rate in MHz
duration = M*P*W*32  # Duration in seconds

# Generate time vector
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)

samples = M*P*W

freq_range = (70, 100)  # Frequency range in MHz

# Generate white noise
white_noise = np.random.normal(0, 1, len(t))

# Design a bandpass filter
nyquist = 0.5 * sample_rate
low = freq_range[0] / nyquist
high = freq_range[1] / nyquist
b, a = butter(4, [low, high], btype='band')


integration_time = 1

band_limited_noise = lfilter(b, a, white_noise)

result_spectrum =  pfb_spectrometer(band_limited_noise, n_taps=M, n_chan=P, n_int=n_int, window_fn="hamming")

In [ ]:
from numpy.fft import fft, rfftfreq
print(result_spectrum.shape)
freqs_band_limited_noise = rfftfreq(P*2-1, 1 / sample_rate)
print(freqs_band_limited_noise.shape)

In [ ]:
plt.plot(freqs_band_limited_noise, db(result_spectrum[0]))

In [ ]:
plt.imshow(db(result_spectrum), cmap='viridis', aspect='auto')
plt.colorbar()
plt.xlabel("Channel")
plt.ylabel("Time")
plt.show()

In [ ]:
result_spectrum_long =  pfb_spectrometer(band_limited_noise, n_taps=M, n_chan=P, n_int=199996//1024, window_fn="hamming")
plt.plot(db(result_spectrum[0]))
plt.plot(db(result_spectrum_long[0]))

In [ ]:
plt.imshow(db(result_spectrum_long), cmap='viridis', aspect='auto')
plt.colorbar()
plt.xlabel("Channel")
plt.ylabel("Time")
plt.show()

In [ ]:
def pfb_spectrometer_walsh(x, n_taps, n_chan, n_int, window_fn="hamming"):
    M = n_taps
    P = n_chan
    
    # Generate window coefficients
    win_coeffs = generate_win_coeffs(M, P, window_fn)

    
    #extend the number of samples for the walsh function to match the signal length 
    print(walsh_matrix.shape)
    period_base = x.shape[0]/32
    walshextended = np.zeros((32,x.shape[0]))
    print(walshextended.shape)
    for i in range(32):
    n= 0
    n2 = period_base
    for j in range(32):
        extended = np.array(walsh_matrix[i][j].repeat(period_base))
        walshextended[i][n:n2] = extended
        n += period_base
        n2 += period_base
    
    #apply walsh function 
    modulated = walshextended*x
    
    # Apply frontend, take FFT, then take power (i.e. square)
    x_fir = pfb_fir_frontend(x, win_coeffs, M, P)
    x_pfb = fft(x_fir, P)
    #x_psd = np.abs(x_pfb)**2
    
    # Trim array so we can do time integration
    x_psd = x_pfb[:np.round(x_psd.shape[0]//n_int)*n_int]
    
    # Integrate over time, by reshaping and summing over axis (efficient)
    x_psd = x_psd.reshape(x_psd.shape[0]//n_int, n_int, x_psd.shape[1])
    x_psd = x_psd.mean(axis=1)
    
    return x_psd


In [ ]:
result_spectrum_walsh =  pfb_spectrometer(band_limited_noise, n_taps=M, n_chan=P, n_int=1, window_fn="hamming")

In [ ]:
plt.imshow(db(result_spectrum_walsh), cmap='viridis', aspect='auto')
plt.colorbar()
plt.xlabel("Channel")
plt.ylabel("Time")
plt.show()

In [ ]:
plt.plot(db(result_spectrum_walsh[0]))